In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
# from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
# from keras.models import Model
# from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
# from keras.optimizers import RMSprop
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing import sequence
# from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import nltk
import nltk as nlp
import string
import re

In [23]:
df_vacancies_parsed_cleared = pd.read_csv('vacancies_parsed_cleared.csv')
df_vacancies_parsed_cleared.head()

,vacancy_id,section_id,section_title,section_body
0,39564357,0,NaN,ооо рсхб интех российская...
1,39564357,1,чем предстоит заниматься,разработка приложений на языке java jakarta e ...
2,39564357,1,чем предстоит заниматься,применение rest подхода к разработке api
3,39564357,1,чем предстоит заниматься,разработка и модификация структур баз данных s...
4,39564357,1,чем предстоит заниматься,работа c предметно ориентированными средами ра...


In [24]:
df_to_mark_up = df_vacancies_parsed_cleared[['section_title']].dropna().drop_duplicates()
df_to_mark_up['target'] = 0
df_to_mark_up.reset_index(drop=True, inplace=True)
df_to_mark_up.head()

,section_title,target
0,чем предстоит заниматься,0
1,мы ждем от вас,0
2,знание систем класса решений,0
3,что мы предлагаем,0
4,требования,0


проанализировав первые 300 элементов датасета я выделил вот такие классы
section_title target<br>
0 чем предстоит заниматься/ключевые задачи<br>
1 требования/мы ждем от вас<br>
2 желательно<br>
3 что мы предлагаем/почему мы/общее описание команды/плюшки от компании<br>
4 условия работы<br>
5 текущий стек<br>
6 все остальное<br>

In [25]:
df_to_mark_up = df_to_mark_up[:300]
df_to_mark_up.to_csv('train_dataset_unmarked.csv')

In [26]:
df_train = pd.read_csv('train_dataset_300.csv', index_col=0)
df_train.head()

,section_title,target
0,чем предстоит заниматься,0
1,мы ждем от вас,1
2,знание систем класса решений,1
3,что мы предлагаем,3
4,требования,1


In [27]:
df_train_outliers = df_train[df_train['target'] == 6]
df_train_outliers.head()

,section_title,target
6,для начала нас ждет еще один переезд на этот ...,6
13,каждый процесс компании имеет своё отражение в...,6
14,1 отлаженные процессы разработки,6
15,2 проработанная архитектура dwh и etl процессов,6
17,4 свежий софт,6


In [28]:
df_train_numpy = df_train.to_numpy()
x = df_train_numpy[:, 0]
y = df_train_numpy[:, 1].astype('int')
counts = np.bincount(y)
print('Most common class has {} of {} total elements. So baseline is {}%'.format(np.max(counts), np.sum(counts), round(np.max(counts)/ np.sum(counts) * 100, 2)))
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=2020)

Most common class has 89 of 300 total elements. So baseline is 29.67%


In [35]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', KNeighborsClassifier(n_neighbors = 7,weights = 'distance',algorithm = 'brute'))])

model_knn = pipe.fit(x_train, y_train)
prediction = model_knn.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 71.67%


In [36]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_knn, x_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.62 (+/- 0.08)


In [37]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model_log = pipe.fit(x_train, y_train)
prediction = model_log.predict(x_test)
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

accuracy: 76.67%


In [38]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_log, x_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.59 (+/- 0.14)


In [39]:
df_full_filled = df_vacancies_parsed_cleared.copy().dropna()
df_full_filled_numpy = df_full_filled.to_numpy()
full_prediction = model_log.predict(df_full_filled_numpy[:, 2])
df_full_filled['target'] = full_prediction
print('{}/{} rows remained in dataset, others were deleted because of NaNs'.format(len(df_full_filled), len(df_vacancies_parsed_cleared)))

41318/51305 rows remained in dataset, others were deleted because of NaNs


In [40]:
df_full_filled.to_csv('vacancies_sections_filled.csv')